In [1]:
"""
# 전처리 배분

1(성준) - 0~9,   40~49, 80~89, 120, 
       - credit_card_balance.csv
       - POS_CASH_balance.csv
       - previous_application.csv
       - installments_payments.csv
2(장호) - 10~19, 50~59, 90~99, 121
3(두영) - 20~29, 60~69, 100~109, 122, 
       - bureau.csv,
4(혜빈) - 30~39, 70~79, 110~119, 123, 
       - bureau_balance.csv
5(준혁) -  
"""

"""
# 데이터 목록
application_test.csv
application_train.csv
credit_card_balance.csv
POS_CASH_balance.csv
previous_application.csv
installments_payments.csv
bureau.csv
bureau_balance.csv
sample_submission.csv
"""

'\n# 데이터 목록\napplication_test.csv\napplication_train.csv\nPOS_CASH_balance.csv\nbureau.csv\nbureau_balance.csv\ncredit_card_balance.csv\ninstallments_payments.csv\nprevious_application.csv\nsample_submission.csv\n'

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
import user_functions as uf

import platform
import matplotlib
%matplotlib inline
# 한글 출력되게 설정
uf.korean()

## previous_application.csv
---

- <span style="color:red">ID 값 중복 존재함</span>

In [95]:
prev_app = pd.read_csv(uf.get_f_path()+"previous_application.csv")

In [3]:
prev_app.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670214 entries, 0 to 1670213
Data columns (total 37 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   SK_ID_PREV                   1670214 non-null  int64  
 1   SK_ID_CURR                   1670214 non-null  int64  
 2   NAME_CONTRACT_TYPE           1670214 non-null  object 
 3   AMT_ANNUITY                  1297979 non-null  float64
 4   AMT_APPLICATION              1670214 non-null  float64
 5   AMT_CREDIT                   1670213 non-null  float64
 6   AMT_DOWN_PAYMENT             774370 non-null   float64
 7   AMT_GOODS_PRICE              1284699 non-null  float64
 8   WEEKDAY_APPR_PROCESS_START   1670214 non-null  object 
 9   HOUR_APPR_PROCESS_START      1670214 non-null  int64  
 10  FLAG_LAST_APPL_PER_CONTRACT  1670214 non-null  object 
 11  NFLAG_LAST_APPL_IN_DAY       1670214 non-null  int64  
 12  RATE_DOWN_PAYMENT            774370 non-nu

In [96]:
# "RATE_DOWN_PAYMENT" 부채 감액 비율
## 음수 값 0으로 대치
prev_app.loc[prev_app["RATE_DOWN_PAYMENT"] < 0, "RATE_DOWN_PAYMENT"] = 0 
## 결측치 0으로 대체
prev_app["RATE_DOWN_PAYMENT"].fillna(0, inplace=True)
## 감액율 중 최대값을 활용해서 새로운 DF 생성
merge_down_rate = prev_app[["SK_ID_CURR", "RATE_DOWN_PAYMENT"]].groupby("SK_ID_CURR").max()

# "AMT_DOWN_PAYMENT" 부채 감액 총액
## 결측치 0으로 대체
prev_app["AMT_DOWN_PAYMENT"].fillna(0, inplace=True)
## 감액총액을 새로운 DF로 생성
merge_down_amt = prev_app[["SK_ID_CURR", "AMT_DOWN_PAYMENT"]].groupby("SK_ID_CURR").sum()

# 연체 횟수 피처 생성
## 부채감액 비율 또는 부채 감액 총액이 0보다 크면 연체라고 판단함
temp = ((prev_app["RATE_DOWN_PAYMENT"] > 0) \
        | (prev_app["AMT_DOWN_PAYMENT"] > 0)
       )
prev_app["연체_횟수"] = temp.replace({True: 1, False: 0})
## 연체 횟수를 더한 새로운 DF 생성
merge_down_num = prev_app[["SK_ID_CURR", "연체_횟수"]].groupby("SK_ID_CURR").sum()

In [118]:
# AMT_CREDIT(대출 승인 금액) 결측치 1개 0으로 대체
prev_app.loc[prev_app["AMT_CREDIT"].isna(), "AMT_CREDIT"] = 0

# AMT_APPLICATION(대출 요청 금액) 0인 행 제외
drop_index = prev_app.loc[prev_app["AMT_APPLICATION"] == 0].index
prev_app.drop(index=drop_index, inplace=True)

# "과거_승인액/요청액" feature 생성
prev_app["과거_승인액/요청액"] = prev_app["AMT_CREDIT"] / prev_app["AMT_APPLICATION"]

# 2.5 이상값은 이상치로 판단하여 모두 2.5로 제한함
## 329개
prev_app.loc[prev_app["과거_승인액/요청액"] > 2.5, "과거_승인액/요청액"] = 2.5

# groupby.mean
## 0인 행을 모두 삭제했으므로 요청액 대비 승인액의 비율의 평균을 통해 한 개인의 대출 요청 금액에 비해 승인액 비율을 도출함
merge_cre_app_r = prev_app[["SK_ID_CURR", "과거_승인액/요청액"]].groupby("SK_ID_CURR").mean()

# new_pre_app Dataframe 만들기
new_pre_app = pd.merge(merge_down_amt, 
                       merge_down_rate, 
                       on="SK_ID_CURR", 
                       how="outer"
                      )
new_pre_app = pd.merge(new_pre_app,
                       merge_cre_app_r, 
                       on="SK_ID_CURR", 
                       how="outer",
                      )
new_pre_app = pd.merge(new_pre_app,
                       merge_down_num, 
                       on="SK_ID_CURR", 
                       how="outer"
                      )

# 결측값은 최빈값(1)로 설정함
# uf.check_feature(new_pre_app, "과거_승인액/요청액")
imput_mode = SimpleImputer(strategy="most_frequent")
new_pre_app["과거_승인액/요청액"] = \
    imput_mode.fit_transform(new_pre_app[["과거_승인액/요청액"]])

# 결측값 확인
uf.check_missing_value(new_pre_app)

# 저장
new_pre_app.to_csv("../pre_data/new_previous_application.csv")

결측치 없음!
